## Importa bibliotecas

In [1]:
import pandas as pd

## Carrega dados

In [11]:
filmes = pd.read_csv("movie_lens/movies.csv")

filmes.columns = ['filmeId', 'titulo', 'genero']
filmes.set_index = 'filmeId'
filmes.head()

,filmeId,titulo,genero
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
notas = pd.read_csv("movie_lens/ratings.csv")
notas.columns = ['usuarioId', 'filmeId', 'nota', 'momento']
notas.head()

,usuarioId,filmeId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
notas.describe()

,usuarioId,filmeId,nota,momento
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


## Primeira tentativa de recomendação

In [15]:
total_de_votos = notas.filmeId.value_counts()
total_de_votos.head()

356     329
318     317
296     307
593     279
2571    278
Name: filmeId, dtype: int64

In [13]:
filmes.loc[356]

filmeId                             412
titulo     Age of Innocence, The (1993)
genero                            Drama
Name: 356, dtype: object

In [16]:
filmes['total_de_votos'] = total_de_votos
filmes.head()

,filmeId,titulo,genero,total_de_votos
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN
1,2,Jumanji (1995),Adventure|Children|Fantasy,215.0
2,3,Grumpier Old Men (1995),Comedy|Romance,110.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,52.0
4,5,Father of the Bride Part II (1995),Comedy,7.0


In [18]:
filmes.sort_values("total_de_votos", ascending=False).head()

,filmeId,titulo,genero,total_de_votos
356,412,"Age of Innocence, The (1993)",Drama,329.0
318,360,I Love Trouble (1994),Action|Comedy,317.0
296,338,Virtuosity (1995),Action|Sci-Fi|Thriller,307.0
593,735,Cemetery Man (Dellamorte Dellamore) (1994),Horror,279.0
2571,3439,Teenage Mutant Ninja Turtles II: The Secret of...,Action|Children|Fantasy,278.0
